# Real-Time Inferencing Service in Azure ML

Цель лабораторной работы: 

- развертывание обученнной ML модели, как __web-сервиса, работающего в режиме реального времени__ (Real-Time Inferencing Service)
- подключение и получение прогнозов от web-сервиса, работающего в режиме реального времени.

## Подготовка среды

Импорт необходимых модулей и проверка версии AzureML SDK:

In [1]:
import azureml.core
from azureml.core import Workspace, Model

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.12.0


Зададим параметры Эксперимента:

In [2]:
experiment_name = 'realtime_service_demo'

experiment_dir = 'realtime-service-demo'
os.makedirs(experiment_dir, exist_ok=True)

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [3]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: ai-in-cloud-workspace.


## Подготовка к развертыванию web-сервиса

Процесс развертывания включает в себя следующие шаги:

- Определите __inference-конфигурацию сервиса__, которая включает скрипт для прогноза и конфигурацию Среды и используемую ML моделm.
- Определите __конфигурацию развертывания__, которая определяет среду выполнения, в которой будет размещен web-сервис. В данном случае это служба [экземпляров контейнеров Azure](https://docs.microsoft.com/ru-ru/azure/container-instances/container-instances-overview).
- Разверните модель как веб-сервис.
- Проверьте состояние развернутого веб-сервиса.

### Получение ML модели

Получим список уже обученных и зарегистрированных в Azure ML моделей машинного обучения:

In [8]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print(f'\t {tag_name}: {tag}')
        
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print(f'\t {prop}: {prop_name}')
        
    print('\n')

diabetes_predict_model v2
	 Demo: ML Pipeline


diabetes_predict_model v1
	 Demo: Target compute
	 0.846851712258014: AUC
	 0.7788888888888889: Accuracy


diabetes_model v4
	 Dataset: Diabetes
	 0.846851712258014: AUC
	 0.7788888888888889: Accuracy


diabetes_model v3
	 Dataset: Diabetes
	 0.846851712258014: AUC
	 0.7788888888888889: Accuracy


diabetes_model v2
	 Dataset: Diabetes
	 0.8468519356081545: AUC
	 0.7788888888888889: Accuracy


diabetes_model v1
	 Training context: Estimator
	 0.8468519356081545: AUC
	 0.7788888888888889: Accuracy


amlstudio-covid19-service v1
	 CreatedByAMLStudio: true


amlstudio-covid19-service-pipe v1
	 CreatedByAMLStudio: true


amlstudio-covid19-spread-servi v1
	 CreatedByAMLStudio: true


amlstudio-pima-diabets-service v2
	 CreatedByAMLStudio: true


amlstudio-letter-recognition-s v1
	 CreatedByAMLStudio: true


amlstudio-pima-diabetes-model v1
	 CreatedByAMLStudio: true




Выберем ML модель, которую хотим развернуть как web-сервис (по умолчанию берется последняя версия модели):

In [10]:
model = ws.models['diabetes_predict_model']
print(f'{model.name} v{model.version}')

diabetes_predict_model v2


## Скрипт для прогнозов

Создадим скрипт, который содержит методы необходимые для:

- `init()`: инициализации web-сервиса
- `run(raw_data)`: прогноза на новых данных.


In [46]:
%%writefile scripts/score_model.py
import os
import json
import joblib
import numpy as np

from azureml.core.model import Model


def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION).
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)


def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)


Overwriting scripts/score_model.py


Скопируем скрипт в директорию эксперимента:

In [47]:
!cp scripts/score_model.py $experiment_dir
!ls $experiment_dir

score_model.py	service_env.yml


### Среда для запуска web-сервиса

Выгрузим Среду, содержащую все необходимые для ML модели зависимости, в файл конфигурации `service_env.yml`, и выведем содержимое этого файла на экран:

In [34]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
service_env = CondaDependencies()
service_env.add_conda_package('scikit-learn')

# Save the environment config as a .yml file
service_env_config = f'{experiment_dir}/service_env.yml'
with open(service_env_config, 'w') as f:
    f.write(service_env.serialize_to_string())
    
print(f'Dependecies was saved to {service_env_config}')

# Print the .yml file
with open(service_env_config, 'r') as f:
    print(f.read())

Dependecies was saved to realtime-service-demo/service_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



## Развертывание web-сервиса 



In [48]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig


service_name = 'diabetes-predict-service'

inference_config = InferenceConfig(runtime='python',
                                   source_directory=experiment_dir,
                                   entry_script='score_model.py',
                                   conda_file='service_env.yml')

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)


service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(f'Service state: {service.state}')

Running..........................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [ ]:
print(service.get_logs())

Если Вам нужно внести изменения и выполнить повторное развертывание, то Вам может потребоваться удалить неработоспособную службу, используя следующий код:
    
`service.delete()`

Найдите на портале Azure ML на вкладке Конечных точек (Endpoints) успешно развернутый web-сервис. Вы также можете получить имена веб-сервисов, выполнив следующий код:

In [50]:
for webservice_name in ws.webservices:
    print(webservice_name)

diabetes-predict-service
covid19-service


In [45]:
# service.delete()

## Подключение к сервису

Подключимся к сервису и получим прогноз по новым данным:

In [54]:
import json

new_data = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print('Patient: {}'.format(new_data[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({'data': new_data})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data=input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
not-diabetic


## Удаление сервиса

Удалите web-сервис в случае отсутствия необходмости в нем, чтобы не тратить лишние средства по подписке Azure, выполнив следующий код:

In [55]:
# service.delete()

## Вывод

## Полезные ссылки

1. https://docs.microsoft.com/ru-ru/azure/machine-learning/how-to-deploy-and-where?tabs=azcli
2. https://docs.microsoft.com/ru-ru/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#choose-a-compute-target 